In [1]:
from plio.io import io_controlnetwork
from knoten.csm import create_csm
import ale
import csmapi
import os
from itertools import product

/home/arsanders/miniconda3/envs/knoten/lib/python3.7/site-packages/ale/__init__.py:22: UserWarning: ALESPICEROOT environment variable not set, Spice Drivers will not function correctly
  warnings.warn('ALESPICEROOT environment variable not set, Spice Drivers will not function correctly')


In [2]:
os.environ["ISISROOT"] = "/usgs/pkgs/isis3.7.0/install"
os.environ["ISIS3DATA"] = "/usgs/cpkgs/isis3/data"
import pysis
from pysis import isis

In [3]:
df = io_controlnetwork.from_isis('data/hand_dense.net')

In [11]:
df

Index(['id', 'pointType', 'pointChoosername', 'pointDatetime', 'pointEditLock',
       'pointIgnore', 'pointJigsawRejected', 'referenceIndex',
       'aprioriSurfPointSource', 'aprioriSurfPointSourceFile',
       'aprioriRadiusSource', 'aprioriRadiusSourceFile', 'latitudeConstrained',
       'longitudeConstrained', 'radiusConstrained', 'aprioriX', 'aprioriY',
       'aprioriZ', 'aprioriCovar', 'adjustedX', 'adjustedY', 'adjustedZ',
       'adjustedCovar', 'pointLog', 'serialnumber', 'measureType', 'sample',
       'line', 'sampleResidual', 'lineResidual', 'measureChoosername',
       'measureDatetime', 'measureEditLock', 'measureIgnore',
       'measureJigsawRejected', 'diameter', 'apriorisample', 'aprioriline',
       'samplesigma', 'linesigma', 'measureLog'],
      dtype='object')

In [5]:
isd_files = []
sensors = {}
for line in open('data/cubes.lis'):
    basename = os.path.splitext(os.path.basename(line.strip()))[0]
    isd = os.path.join('data',basename+'.json')
    isd_files.append(isd)
    with open(isd, 'w+') as f:
        f.write(ale.loads(line.strip(), formatter='usgscsm'))
        
    sn = isis.getsn(from_=line.strip()).strip().decode('utf-8')
    sensors[sn] = create_csm(isd)


In [9]:
def computeSensorPartials(sensors, control_network):
    partials = []
    ground_points = [csmapi.EcefCoord(x, y, z) for 
                     x, y, z in zip(control_network['adjustedX'],
                                    control_network['adjustedY'],
                                    control_network['adjustedZ'])]

    partials.append([sensor.computeSensorPartials(p_idx, gp) for 
                     p_idx, gp in product(range(sensor.getNumParameters()), ground_points)])
    return partials
        
def computeGroundPartials(sensors, control_network):
    partials = []
    for sn, sensor in sensors.items():
        ground_points = [csmapi.EcefCoord(x, y, z) for 
                            x, y, z in zip(control_network['adjustedX'],
                                            control_network['adjustedY'],
                                            control_network['adjustedZ'])]
        
        partials.append([sensor.computeGroundPartials(gp) for 
                         gp in ground_points])
    return partials

In [10]:
sensor_partials = computeSensorPartials(sensors, df)
print(len(sensor_partials))
print(len(sensor_partials[0]))
print(sensor_partials[0][0])

ground_partials = computeGroundPartials(sensors, df)
print(len(ground_partials))
print(len(ground_partials[0]))
print(ground_partials[0][0])

4
2080
<Swig Object of type 'std::pair< double,double > *' at 0x7fc83da73450>
4
130
<Swig Object of type 'std::vector< double > *' at 0x7fc83da52360>


In [ ]:
def computeSensorPartials(sensors, control_network):
    partials = []
    for sn, sensor in sensors.items():
        ground_points = [csmapi.EcefCoord(x, y, z) for 
                         x, y, z in zip(control_network['adjustedX'],
                                        control_network['adjustedY'],
                                        control_network['adjustedZ'])]
        
        partials.append([sensor.computeSensorPartials(p_idx, gp) for 
                         p_idx, gp in product(range(sensor.getNumParameters()), ground_points)])
    return partials
        
def computeGroundPartials(sensors, control_network):
    partials = []
    for sn, sensor in sensors.items():
        ground_points = [csmapi.EcefCoord(x, y, z) for 
                            x, y, z in zip(control_network['adjustedX'],
                                            control_network['adjustedY'],
                                            control_network['adjustedZ'])]
        
        partials.append([sensor.computeGroundPartials(gp) for 
                         gp in ground_points])
    return partials